In [1]:
from tqdm.notebook import tqdm, trange
import pandas as pd
import numpy as np
import torch as pt
import matplotlib.pyplot as plt
import seaborn as sns
import pyqtgraph as pg
import pyqtgraph.opengl as gl

from calibrators import Calibrator
from calibrators.models import Complex
from calibrators.models.gradients.learn import SqDec

In [2]:
data = {
    'good': pd.read_csv('./data_archive/good_readings.csv'),
    'okay': pd.read_csv('./data_archive/okay_readings.csv'),
    'bad': pd.read_csv('./data_archive/bad_readings.csv'),
    'ref': pd.Series([17.8480, 4.5939, 50.8366], index=['mag_x','mag_y','mag_z'])
}

In [ ]:
data_adj = {}
coefs = {}
calibs = {}
for key, val in data.items():
    if key == 'ref': continue
    calibs[key] = Calibrator(
        val.values, data['ref'],
        Model=Complex, model_params={'gradient_params':{'Learn':SqDec}},
        verbose=True
    )
    data_adj[key] = calibs[key].calibrate(100000)
    coefs[key] = calibs[key].coefs

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
coefs['good']

In [ ]:
coefs['okay']

In [ ]:
coefs['bad']

In [ ]:
print('good:', np.sqrt((data_adj['good']**2).sum(axis=1)).mean())
print('okay:', np.sqrt((data_adj['okay']**2).sum(axis=1)).mean())
print('bad:', np.sqrt((data_adj['bad']**2).sum(axis=1)).mean())
print('ref:', np.sqrt((data['ref']**2).sum()))

In [ ]:
plt.figure(figsize=(16,10))
sns.distplot(np.sqrt((data_adj['good']**2).sum(axis=1)), label='good')
sns.distplot(np.sqrt((data_adj['okay']**2).sum(axis=1)), label='okay')
sns.distplot(np.sqrt((data_adj['bad']**2).sum(axis=1)), label='bad')
plt.axvline(np.sqrt((data['ref']**2).sum()), color='#000000', label='ref')
plt.legend()